In [75]:
import pandas as pd
df=pd.read_csv("FuelConsumption.csv")
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,2014,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,2014,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,2014,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


In [76]:
from sklearn.model_selection import train_test_split,GridSearchCV
train,test=train_test_split(df,test_size=.2,random_state=52)
x_train=train[["ENGINESIZE","CYLINDERS","FUELCONSUMPTION_COMB","FUELTYPE"]]
x_test=test[["ENGINESIZE","CYLINDERS","FUELCONSUMPTION_COMB","FUELTYPE"]]

In [77]:
numerical_cols=["ENGINESIZE","CYLINDERS"]
categorical_col=["FUELTYPE"]
y_train=train["CO2EMISSIONS"]
y_test=test["CO2EMISSIONS"]

In [78]:
from sklearn.preprocessing import OneHotEncoder


In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler
Preprocess=ColumnTransformer([
    ('num',Pipeline([
        ('polyfeatures',PolynomialFeatures(degree=2)),
        ('scale',MinMaxScaler())
        ]),numerical_cols),
    ('cat',OneHotEncoder(),categorical_col)
])

model=DecisionTreeRegressor(random_state=42)
mypipeline=Pipeline(steps=[('preprocess',Preprocess),('mymodel',  model)])
mypipeline.fit(x_train,y_train)



Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('polyfeatures',
                                                                   PolynomialFeatures()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['ENGINESIZE', 'CYLINDERS']),
                                                 ('cat', OneHotEncoder(),
                                                  ['FUELTYPE'])])),
                ('mymodel', DecisionTreeRegressor(random_state=42))])

In [1]:
mysearch=GridSearchCV(estimator=mypipeline,param_grid={"max_depth":[4,5,6,7,10,12,14,16,18,20,30,40,50,100],"min_samples_leaf":[1,3,5,7,9,10]})

mysearch.fit(x_train,y_train)


NameError: name 'GridSearchCV' is not defined

In [93]:
mysearch.best_params_

{'mymodel__max_depth': 7, 'mymodel__min_samples_leaf': 3}

In [98]:
model=DecisionTreeRegressor(random_state=42,max_depth=7,min_samples_leaf=3)
mypipeline=Pipeline(steps=[('preprocess',Preprocess),('mymodel',  model)])
mypipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('polyfeatures',
                                                                   PolynomialFeatures()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['ENGINESIZE', 'CYLINDERS']),
                                                 ('cat', OneHotEncoder(),
                                                  ['FUELTYPE'])])),
                ('mymodel',
                 DecisionTreeRegressor(max_depth=7, min_samples_leaf=3,
                                       random_state=42))])

In [99]:
from sklearn.metrics import mean_squared_error
y_train_=mypipeline.predict(x_train)
mse=mean_squared_error(y_train,y_train_)
print("rmse on train data is:",mse**.5)

rmse on train data is: 23.67678301104321


In [101]:
from sklearn.metrics import mean_squared_error
y_test_=mypipeline.predict(x_test)
mse=mean_squared_error(y_test,y_test_)
print("rmse on test data is:",mse**.5)

rmse on test data is: 28.638771113752533


In [102]:
from sklearn.metrics import r2_score
y_test_=mypipeline.predict(x_test)
r2=r2_score(y_test,y_test_)
print("r2_score on test data is:",r2)

r2_score on test data is: 0.8093347496631023
